In [1]:
import matplotlib
import matplotlib.pyplot as plt
import gzip, pickle
import numpy as np
from tsCluster import tsCluster as tsc
from copy import deepcopy
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from notebooks.helper import get_onehot_codes
import pandas as pd
import random
import pyriemann
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.decomposition import PCA
import os
import seaborn as sn
import pandas as pd
import re
import scipy

#
from sklearn.cluster import KMeans

#pip install pyriemann
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace
from pyriemann.utils.distance import distance_riemann

from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.metrics import plot_confusion_matrix, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity

/anaconda3/envs/dodtin/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.cluster.k_means_ module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)
/anaconda3/envs/dodtin/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
matplotlib.__version__

'3.0.3'

In [3]:
def get_label_4_old(str_inp):
    if str_inp == 'ctr':
        return 0
    if str_inp == 'hl':
        return 1
    if str_inp == 'tin_hl':
        return 2
    if str_inp == 'tin':
        return 3

In [4]:
def get_label_8(str_inp,loc):
    if loc == 'chm':
        to_add = 0
    elif loc == 'san':
        to_add = 1
    else:
        raise ValueError("Invalid location")
    
    if str_inp == 'ctr':
        return 0 + to_add
    if str_inp == 'hl':
        return 1 + to_add
    if str_inp == 'tin_hl':
        return 2 + to_add
    if str_inp == 'tin':
        return 3 + to_add

In [5]:
def get_label_4(str_inp):
    return str_inp

In [6]:
def get_label_4_tin_thresh(str_inp,name,metadata):
    if str_inp == 'ctr':
        return 0
    if str_inp == 'hl':
        return 1
    if str_inp == 'tin_hl':
        if metadata.loc[int(re.sub(r'sub_','',name)),'TFI_A'] <= 25:
            return 2
        else:
            return 3
    if str_inp == 'tin':
        if metadata.loc[int(re.sub(r'sub_','',name)),'TFI_A'] <= 25:
            return 4
        else:
            return 5
        

In [7]:
def get_label_4_tin_thresh2(str_inp,name,metadata):
    if str_inp == 'ctr':
        return 0
    if str_inp == 'hl':
        return 1
    if str_inp == 'tin_hl' or str_inp == 'tin':
        if metadata.loc[int(re.sub(r'sub_','',name)),'TFI_A'] <= 25:
            return 2
        else:
            return 3
        

## Create result directories

In [8]:
try:
    os.mkdir('RiemannResults')
except FileExistsError:
    pass

In [9]:
data_dir = 'bin_data/'
fnames = ['wgsr_data33_tv_linear', 'nogsr_data33_tv_linear', 'wgsr_band33_tv_linear', 'nogsr_band33_tv_linear', 'wgsr_20band33_tv_linear', 'nogsr_20band33_tv_linear']

for fname in fnames:
    try:
        os.mkdir('RiemannResults/'+fname)
    except FileExistsError:
        pass

## Number of features is larger than number of samples, so linear kernel should be used in svm

In [10]:
valid_runs_pre = pd.read_csv('Valid_Runs.csv',index_col='ID',header=0,names=['ID','A','B','C','D','E','F','G'],usecols=['ID','A','B'])

In [11]:
valid_runs_dict = {re.sub(r'\d+_','',idx.rstrip('*')): row.values for idx,row in valid_runs_pre.iterrows()}

In [12]:
valid_runs_dict.keys()

dict_keys(['113', '115', '117', '120', '122', '123', '126', '166', '167', '168', '169', '199', '200', '220', '258', '259', '261', '275', '125', '189', '191', '201', '226', '230', '243', '253', '265', '270', '271', '273', '284', '285', '103', '130', '135', '136', '150', '157', '182', '216', '218', '233', '274', '276', '286', '287', '291', '292', '102', '105', '106', '108', '129', '131', '133', '140', '146', '152', '153', '154', '158', '160', '162', '163', '174', '176', '178', '184', '187', '188', '197', '205', '207', '208', '210', '213', '217', '219', '222', '223', '227', '246', '247', '255', '256', '257', '277', '279', '280', '282', '293'])

In [13]:
metadata_1 = pd.read_csv('metadata_py/UIUC-Table 1.csv',index_col = 'ID')
metadata_2 = pd.read_csv('metadata_py/WHASC-Table 1.csv',index_col = 'ID')

FileNotFoundError: File b'metadata_py/UIUC-Table 1.csv' does not exist

In [15]:
metadata = pd.concat([metadata_1,metadata_2])

In [16]:
with gzip.open(data_dir+fnames[0] + '.bin', 'rb') as stream:
        data = pickle.load(stream)

In [17]:
data.samples[0]['Name']

'sub_257'

In [18]:
data.samples[0]['Run']

'run1'

In [19]:
#[x['Location'] for x in data.samples]

In [20]:
def data_selector(sample):
    cond1 = int(sample['Run'].lstrip('run')) in valid_runs_dict.get(re.sub(r'sub_','',sample['Name']),[])
    cond2 = int(sample['Run'].lstrip('run')) <= 2
    return cond1 and cond2

In [21]:
from sklearn.base import BaseEstimator, TransformerMixin
class PCA_coord_change(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.pca_instance = PCA()
        
    def fit(self,X,y=None):
        
        self.pca_instance.fit(X,y)
        return self
    
    def transform(self,X):
        shape = X.shape
        coords = self.pca_instance.components_
        coord_shape = coords.shape
        if coord_shape[1] < shape[0]:
            coords_padded = np.hstack([coords, np.zeros((coord_shape[0],-coord_shape[1]+shape[0]))])
            X_padded = X
        else:
            coords_padded = coords
            X_padded = np.vstack([X,np.zeros((coord_shape[1]-shape[0],shape[1]))])
        return np.dot(coords_padded,X_padded)
       
    
    def fit_transform(self,X,y=None,sample_weight=None):
        
        self.pca_instance.fit(X,y)
        shape = X.shape
        coords = self.pca_instance.components_
        coord_shape = coords.shape
        if coord_shape[1] < shape[0]:
            coords_padded = np.hstack([coords, np.zeros((coord_shape[0],-coord_shape[1]+shape[0]))])
            X_padded = X
        else:
            coords_padded = coords
            X_padded = np.vstack([X,np.zeros((coord_shape[1]-shape[0],shape[1]))])
        return np.dot(coords_padded,X_padded)
        
    
class to_upper_tri(BaseEstimator, TransformerMixin):
    def __init__(self,k):
        self.k = k
        
    def fit(self,X,y=None):
        return self
    
    def transform(self,X):
        shape = X[0].shape
        inds = np.triu_indices(shape[0],self.k)
        return np.array([x[inds].flatten() for x in X])
    
    def fit_transform(self,X,y=None,sample_weight=None):
        shape = X[0].shape
        inds = np.triu_indices(shape[0],self.k)
        return np.array([x[inds].flatten() for x in X])

In [ ]:
accDict = {}
simDict = {}
matDict = {}
corrDict = {}
spearDict = {}
for fname in fnames:
    with gzip.open(data_dir+fname + '.bin', 'rb') as stream:
        data = pickle.load(stream)
        
    simArr = []
        
    # get time series data to make covariance matrices
    X = np.array([sample['TimeSeries'] for sample in data.samples if data_selector(sample)])        
    y = np.array([get_label_4_old(sample['Group']) for sample in data.samples if data_selector(sample)])
    
    # gsr seems to produce a rank deficient covariance matrix, so oas regularization is necessary
    if fname.startswith('no'):
        covest = Covariances()
    else:
        covest = Covariances('oas')
    ts = TangentSpace()
    svc = SVC(kernel='linear')
    clf_riem = make_pipeline(covest,ts,svc)
    
    if fname.startswith('no'):
        covest2 = Covariances()
    else:
        covest2 = Covariances('oas')
    svc2 = SVC(kernel='linear')
    get_tri_inds = to_upper_tri()
    clf_cov = make_pipeline(covest2,get_tri_inds,svc2)
        
    X_base = np.array([sample['CRM'][np.triu_indices(sample['CRM'].shape[0],1)].flatten() for sample in data.samples if data_selector(sample)])
    y_base = np.array([get_label_4_old(sample['Group']) for sample in data.samples if data_selector(sample)])
    svc_base = SVC(kernel='linear')
    clf_base = svc_base
    
    svc_pca = SVC(kernel='linear')
    decomp_pca = PCA_coord_change()
    clf_pca = make_pipeline(decomp_pca,svc_pca)
    
    X_lag = np.array([sample['LCM'][np.triu_indices(sample['LCM'].shape[0],1)].flatten() for sample in data.samples if data_selector(sample)])
    y_lag = np.array([get_label_4_old(sample['Group']) for sample in data.samples if data_selector(sample)])
    svc_lag = SVC(kernel='linear')
    clf_lag = svc_lag
    
    clf_composite = SVC(kernel='linear')
    
    #Check clustering
    #to_TS = make_pipeline(covest,ts)
    #X_in_TS = to_TS.transform(X)
    #kmeans = KMeans(n_clusters=4,random_state=0).fit(X_in_TS)
            
    # Monte Carlo, in theory should run this len(y)^2 times, but I need to save my poor computer's memory.
    accRiemList = []
    accCovList = []
    accPcaList = []
    accBaseList = []
    accLagList = []
    accCompList = []
    coeffArr = []
    matRiemList = []
    matBaseList = []
    matLagList = []
    corrArrBefore = []
    corrArrAfter = []
    spearArrBefore = []
    spearArrAfter = []

    rs = StratifiedShuffleSplit(n_splits=100, test_size=.3)
    for i,(train_inds,test_inds) in enumerate(rs.split(X,y)):
        
        X_train, X_test, y_train, y_test = X[train_inds],X[test_inds],y[train_inds],y[test_inds]
        X_train_cov, X_test_cov, y_train_cov, y_test_cov = X_train.copy(), X_test.copy(), y_train.copy(), y_test.copy()
        X_train_base, X_test_base, y_train_base, y_test_base = X_base[train_inds],X_base[test_inds],y_base[train_inds],y_base[test_inds]
        X_train_lag, X_test_lag, y_train_lag, y_test_lag = X_lag[train_inds],X_lag[test_inds],y_lag[train_inds],y_lag[test_inds]
        
        clf_riem.fit(X_train,y_train)
        clf_cov.fit(X_train_cov,y_train_cov)
        clf_pca.fit(X_train_base.copy(),y_train_base.copy())
        clf_base.fit(X_train_base,y_train_base)
        clf_lag.fit(X_train_lag,y_train_lag)
        clf_composite.fit(X_train_lag,y_train_lag)
        
        #get riemann svm coefficients
        coeffArr.append(clf_riem[2].coef_)
            
        #compare correlation
        corr_coeffs_before = np.corrcoef(np.vstack([x[np.triu_indices(33)].flatten() for x in X_train]),rowvar=False)
        corrArrBefore.append(np.linalg.norm(corr_coeffs_before))
        #spearman correlation
        spearman_coeffs_before,_ = scipy.stats.spearmanr(np.vstack([x[np.triu_indices(33)].flatten() for x in X_train]),axis=0)
        spearArrBefore.append(np.linalg.norm(spearman_coeffs_before))
        
        ref = ts.reference_
        covs = covest.transform(X_train)
        mapped = ts.transform(covs)
        corr_coeffs_after = np.corrcoef(mapped,rowvar=False)
        spearman_coeffs_after = scipy.stats.spearmanr(mapped,axis=0)
        corrArrAfter.append(np.linalg.norm(corr_coeffs_after))
        spearArrAfter.append(np.linalg.norm(spearman_coeffs_after))
        
        #for flattened in mapped:
            
            #mat = np.zeros((33,33))
            #mat[np.triu_indices(33)] = flattened
            #mat = (mat + mat.T)/2
            #mapped_mats.append(mat)
            
        #print(np.mean(np.array([distance_riemann(x,ref) for x in covs])-np.array([np.linalg.norm(x,ord='fro') for x in mapped_mats])/np.array([distance_riemann(x,ref) for x in covs])))
        
        y_pred = clf_riem.predict(X_test)
        y_pred_cov = clf_cov.predict(X_test_cov)
        y_pred_pca = clf_pca.predict(X_test_base.copy())
        y_pred_base = clf_base.predict(X_test_base)
        y_pred_lag = clf_lag.predict(X_test_lag)
        
        # save accuracy
        accRiemList.append(accuracy_score(y_pred,y_test))
        accCovList.append(accuracy_score(y_pred_cov,y_test_cov))
        accPcaList.append(accuracy_score(y_pred_pca,y_test_base.copy()))
        accBaseList.append(accuracy_score(y_pred_base,y_test_base))
        accLagList.append(accuracy_score(y_pred_lag,y_test_lag))
        accCompList.append(accuracy_score(clf_composite.predict(X_test_base),y_test_base))
        
        # confusion matrix
        mat = confusion_matrix(y_test,y_pred,normalize='true',labels=[0,1,2,3])
        #plot_confusion_matrix(clf_riem, X_test, y_test,normalize='true',labels=['ctr','hl','tin_hl','tin'],display_labels=['ctr','ctr_hl','hl_tin','tin'])
        #plt.savefig('RiemannResults/' + fname + '/ConfMatRiemRun' + str(i) + '.png',bbox_inches='tight')
        #plt.close()
        mat_base = confusion_matrix(y_test_base,y_pred_base,normalize='true',labels=[0,1,2,3])
        #plot_confusion_matrix(clf_base, X_test_base, y_test_base,normalize='true',labels=['ctr','hl','tin_hl','tin'],display_labels=['ctr','ctr_hl','hl_tin','tin'])
        #plt.savefig('RiemannResults/' + fname + '/ConfMatBaseRun' + str(i) + '.png',bbox_inches='tight')
        #plt.close()
        mat_lag = confusion_matrix(y_test_lag,y_pred_lag,normalize='true',labels=[0,1,2,3])
        #plot_confusion_matrix(clf_lag, X_test_lag, y_test_lag,normalize='true',labels=['ctr','hl','tin_hl','tin'],display_labels=['ctr','ctr_hl','hl_tin','tin'])
        #plt.savefig('RiemannResults/' + fname + '/ConfMatLagRun' + str(i) + '.png',bbox_inches='tight')
        #plt.close()
        
        matRiemList.append(mat)
        matBaseList.append(mat_base)
        matLagList.append(mat_lag)
    
    for z in range(0,len(coeffArr[0])):
        class_z_coeffs = [x[z] for x in coeffArr]
        cos_sim = cosine_similarity(class_z_coeffs)
        upperTri = cos_sim[np.triu_indices(cos_sim.shape[0],1)]
        cos_avg = np.mean(upperTri.flatten())
        simArr.append(cos_avg)
        
    avgMatRiem = sum(matRiemList)/len(matRiemList)
    simDict.update({fname: simArr}) 
    matDict.update({fname: avgMatRiem})
    riemAcc = np.mean(accRiemList)
    covAcc = np.mean(accCovList)
    pcaAcc = np.mean(accPcaList)
    baseAcc = np.mean(accBaseList)
    lagAcc = np.mean(accLagList)
    compAcc = np.mean(accCompList)
    accDict.update({fname:{'riem':riemAcc, 'cov':covAcc, 'pca': pcaAcc, 'base': baseAcc,'lag': lagAcc,'comp':compAcc}})
    corrDict.update({fname:{'before':np.mean(corrArrBefore),'after':np.mean(corrArrAfter)}}) 
    spearDict.update({fname:{'before':np.mean(spearArrBefore),'after':np.mean(spearArrAfter)}}) 
    print("Mean Accuracy w/ Riemann on file " + fname + ": " + str(riemAcc))
    print("Mean Accuracy w/ Cov on file " + fname + ": " + str(covAcc))
    print("Mean Accuracy w/ PCA on file " + fname + ": " + str(pcaAcc))
    print("Mean Accuracy w/ Base on file " + fname + ": " + str(baseAcc))
    print("Mean Accuracy w/ Lag on file " + fname + ": " + str(lagAcc))
    print("Mean Accuracy w/ Composite on file " + fname + ": " + str(compAcc))
    print("----------------")


In [ ]:
corrDict

In [ ]:
spearDict

In [ ]:
simDict

In [ ]:
for fname,mat in matDict.items():   
    plt.figure()
    df_cm = pd.DataFrame(mat, index = ['ctr','ctr_hl','hl_tin','tin'],
                      columns = ['ctr','ctr_hl','hl_tin','tin'])

    sn.heatmap(df_cm, annot=True)
    plt.title(fname + " average confusion matrix")
    plt.ylabel('actual label')
    plt.xlabel('predicted label')
    plt.savefig('RiemannResults/'+fname+'/AvgConfMat.png',bbox_inches='tight')

### Divide by sum along rows to normalize by total predictions in a given class 

In [ ]:
for fname,mat in matDict.items():   
    plt.figure()
    df_cm = pd.DataFrame(mat/np.sum(mat,axis=0), index = ['ctr','ctr_hl','hl_tin','tin'],
                      columns = ['ctr','ctr_hl','hl_tin','tin'])

    sn.heatmap(df_cm, annot=True)
    plt.title(fname + " Pr(actual label | predicted label)")
    plt.ylabel('actual label')
    plt.xlabel('predicted label')
    plt.savefig('RiemannResults/'+fname+'/ConditionalAvgConfMat.png',bbox_inches='tight')

### Get discriminative connections

In [22]:
def ind_to_pair(ind):
    x,y = np.triu_indices(33)
    return (x[ind],y[ind])

In [23]:
def get_sig_pairs(data,coeffarr,nullcoeffs,p,maxmin="max"):
    rois = data.rois
    if maxmin == "max":
        percentile = 100-p
    elif maxmin == "min":
        percentile = p
    thresholds = np.percentile(nullcoeffs,percentile,axis=0)
    if maxmin == "max":
        boolarr =  coeffarr > thresholds.reshape(6,1)
    elif maxmin == "min":
        boolarr = coeffarr < thresholds.reshape(6,1)
    indarr = [[(rois[ind_to_pair(x)[0]],rois[ind_to_pair(x)[1]]) for i,x in enumerate(range(0,len(boolarr[0]))) if boolarr[j][i]] for j in range(0,len(boolarr))]
    return indarr

In [25]:
with gzip.open(data_dir+fnames[1] + '.bin', 'rb') as stream:
        data = pickle.load(stream)

# First, make null model
# Use integer labeling so we can be sure that the one vs one classifiers are 
# in the correct orders
X = np.array([sample['CRM'] for sample in data.samples if (sample['Run']=='run1' or sample['Run'] == 'run2')])        
y = np.array([get_label_4_old(sample['Group']) for sample in data.samples if (sample['Run']=='run1' or sample['Run'] == 'run2')])
# Randomly permute labels (only labels, not training input)

NUM_BOOTSTRAP = 10
svc = SVC(kernel='linear')
tut= to_upper_tri(1)
clf_riem = make_pipeline(tut,svc)
maxcoeffs = []   
mincoeffs = []
nullcoeffs = []
nullcos = []
for i in range(0,100):
    y_permuted = np.random.permutation(y)
    coeffArr = []
    rs = ShuffleSplit(n_splits=NUM_BOOTSTRAP, test_size=.3)
    for train,test in rs.split(X):
        X_train, X_test, y_train, y_test = X[train],X[test],y_permuted[train],y_permuted[test]
        clf_riem.fit(X_train,y_train)
        coeffArr.append(clf_riem[1].coef_/np.std(clf_riem[1].coef_,axis=-1).reshape(6,1))
        
    meancoeff = sum(coeffArr)/len(coeffArr)
    classcos = []
    for z in range(0,len(coeffArr[0])):
        class_z_coeffs = [x[z] for x in coeffArr]
        cos_sim = cosine_similarity(class_z_coeffs)
        upperTri = cos_sim[np.triu_indices(cos_sim.shape[0],1)]
        cos_max = np.max(upperTri.flatten())
        classcos.append(cos_max)
        
    nullcos.append(classcos)
    nullcoeffs.append(meancoeff)
    maxcoeff = np.max(meancoeff,axis=-1)
    mincoeff = np.min(meancoeff,axis=-1)
    maxcoeffs.append(maxcoeff)
    mincoeffs.append(mincoeff)
    
coeffArr = []
rs = ShuffleSplit(n_splits=NUM_BOOTSTRAP, test_size=.3)
for train,test in rs.split(X):
    X_train, X_test, y_train, y_test = X[train],X[test],y[train],y[test]
    clf_riem.fit(X_train,y_train)
    coeffArr.append(clf_riem[1].coef_/np.std(clf_riem[1].coef_,axis=-1).reshape(6,1))
meancoeff = sum(coeffArr)/len(coeffArr)
classcos = []
for z in range(0,len(coeffArr[0])):
    class_z_coeffs = [x[z] for x in coeffArr]
    cos_sim = cosine_similarity(class_z_coeffs)
    upperTri = cos_sim[np.triu_indices(cos_sim.shape[0],1)]
    cos_max = np.max(upperTri.flatten())
    classcos.append(upperTri.flatten())
cossims=[classcos]


In [29]:
sig_pairs_max = get_sig_pairs(data,meancoeff,maxcoeffs,10,"max")
sig_pairs_min = get_sig_pairs(data,meancoeff,mincoeffs,10,"min")
cmpr_list = ['ctr_vs_ctr_hl','ctr_vs_tin_hl','ctr_vs_tin','ctr_hl_vs_tin_hl','ctr_hl_vs_tin','tin_hl_vs_tin']
discrim_dict_max = {cmpr_list[i]: sig_pairs_max[i] for i in range(0,6)}
discrim_dict_min = {cmpr_list[i]: sig_pairs_min[i] for i in range(0,6)}

In [30]:
discrim_dict_max

{'ctr_vs_ctr_hl': [],
 'ctr_vs_tin_hl': [],
 'ctr_vs_tin': [('lsuperiortemporaljunction', 'ramygdala')],
 'ctr_hl_vs_tin_hl': [],
 'ctr_hl_vs_tin': [],
 'tin_hl_vs_tin': [('lmidfrontalgyrus', 'ranteriorinsula'),
  ('medialprefrontalcortex', 'rfrontaleyefield')]}

In [31]:
discrim_dict_min

{'ctr_vs_ctr_hl': [('lmidfrontalgyrus', 'lprimaryauditorycortex')],
 'ctr_vs_tin_hl': [('lsuperioroccipitallobe', 'rmidfrontalgyrus')],
 'ctr_vs_tin': [],
 'ctr_hl_vs_tin_hl': [],
 'ctr_hl_vs_tin': [('lamygdala', 'medialprefrontalcortex')],
 'tin_hl_vs_tin': []}

In [32]:
with gzip.open(data_dir+fnames[1] + '.bin', 'rb') as stream:
        data = pickle.load(stream)

# First, make null model
# Use integer labeling so we can be sure that the one vs one classifiers are 
# in the correct orders
X = np.array([sample['TimeSeries'] for sample in data.samples if (sample['Run']=='run1' or sample['Run'] == 'run2')])        
y = np.array([get_label_4_old(sample['Group']) for sample in data.samples if (sample['Run']=='run1' or sample['Run'] == 'run2')])
# Randomly permute labels (only labels, not training input)

NUM_BOOTSTRAP = 10
covest = Covariances()
ts = TangentSpace()
svc = SVC(kernel='linear')
clf_riem = make_pipeline(covest,ts,svc)
maxcoeffs = []   
mincoeffs = []
nullcoeffs = []
nullcos = []
for i in range(0,100):
    y_permuted = np.random.permutation(y)
    coeffArr = []
    rs = ShuffleSplit(n_splits=NUM_BOOTSTRAP, test_size=.3)
    for train,test in rs.split(X):
        X_train, X_test, y_train, y_test = X[train],X[test],y_permuted[train],y_permuted[test]
        clf_riem.fit(X_train,y_train)
        coeffArr.append(clf_riem[2].coef_/np.std(clf_riem[2].coef_,axis=-1).reshape(6,1))
        
    meancoeff = sum(coeffArr)/len(coeffArr)
    classcos = []
    for z in range(0,len(coeffArr[0])):
        class_z_coeffs = [x[z] for x in coeffArr]
        cos_sim = cosine_similarity(class_z_coeffs)
        upperTri = cos_sim[np.triu_indices(cos_sim.shape[0],1)]
        cos_max = np.max(upperTri.flatten())
        classcos.append(cos_max)
        
    nullcos.append(classcos)
    nullcoeffs.append(meancoeff)
    maxcoeff = np.max(meancoeff,axis=-1)
    mincoeff = np.min(meancoeff,axis=-1)
    maxcoeffs.append(maxcoeff)
    mincoeffs.append(mincoeff)
    
coeffArr = []
rs = ShuffleSplit(n_splits=NUM_BOOTSTRAP, test_size=.3)
for train,test in rs.split(X):
    X_train, X_test, y_train, y_test = X[train],X[test],y[train],y[test]
    clf_riem.fit(X_train,y_train)
    coeffArr.append(clf_riem[2].coef_/np.std(clf_riem[2].coef_,axis=-1).reshape(6,1))
meancoeff = sum(coeffArr)/len(coeffArr)
classcos = []
for z in range(0,len(coeffArr[0])):
    class_z_coeffs = [x[z] for x in coeffArr]
    cos_sim = cosine_similarity(class_z_coeffs)
    upperTri = cos_sim[np.triu_indices(cos_sim.shape[0],1)]
    cos_max = np.max(upperTri.flatten())
    classcos.append(upperTri.flatten())
cossims=[classcos]


In [33]:
mean_cossims = [np.mean(x) for x in cossims[0]]

In [34]:
mean_cossims

[0.8354182994766365,
 0.7888007461782324,
 0.8046822291302825,
 0.7821434494688512,
 0.8203338218959366,
 0.7649699962585795]

In [35]:
null_dist = np.array(nullcos).T

In [36]:
null_dist_thresh = [np.percentile(x,97) for x in null_dist]

In [37]:
null_dist_thresh

[0.7865834841596142,
 0.7506129008005149,
 0.7962413453967285,
 0.7677454378167389,
 0.8037777291430522,
 0.7588356045210332]

In [38]:
mean_cossims > null_dist_thresh

True

In [39]:
sig_pairs_max = get_sig_pairs(data,meancoeff,maxcoeffs,5,"max")

In [40]:
sig_pairs_min = get_sig_pairs(data,meancoeff,mincoeffs,5,"min")

In [41]:
cmpr_list = ['ctr_vs_ctr_hl','ctr_vs_tin_hl','ctr_vs_tin','ctr_hl_vs_tin_hl','ctr_hl_vs_tin','tin_hl_vs_tin']
discrim_dict_max = {cmpr_list[i]: sig_pairs_max[i] for i in range(0,6)}

In [43]:
discrim_dict_max

{'ctr_vs_ctr_hl': [('lventralintraparietalsulcus',
   'lventralintraparietalsulcus'),
  ('precuneus', 'rcuneus'),
  ('rcuneus', 'rprimaryvisualcortex')],
 'ctr_vs_tin_hl': [('posteriorcingulatecortex', 'posteriorcingulatecortex'),
  ('rmidfrontalgyrus', 'rsuperiortemporalsulcus')],
 'ctr_vs_tin': [('posteriorcingulatecortex', 'posteriorcingulatecortex')],
 'ctr_hl_vs_tin_hl': [('rcuneus', 'rsuperioroccipitallobe'),
  ('rfrontaleyefield', 'rfrontaleyefield')],
 'ctr_hl_vs_tin': [('lamygdala', 'lamygdala'),
  ('lprimaryvisualcortex', 'lprimaryvisualcortex'),
  ('lprimaryvisualcortex', 'rcuneus')],
 'tin_hl_vs_tin': [('lamygdala', 'rparahippocampus'),
  ('rcuneus', 'rcuneus'),
  ('rinferiorparietallobe', 'rsuperiortemporaljunction')]}

In [44]:
discrim_dict_min = {cmpr_list[i]: sig_pairs_min[i] for i in range(0,6)}

In [45]:
discrim_dict_min

{'ctr_vs_ctr_hl': [('lprimaryvisualcortex', 'lprimaryvisualcortex')],
 'ctr_vs_tin_hl': [('lsuperiortemporaljunction', 'posteriorcingulatecortex'),
  ('rmidfrontalgyrus', 'rmidfrontalgyrus')],
 'ctr_vs_tin': [],
 'ctr_hl_vs_tin_hl': [],
 'ctr_hl_vs_tin': [],
 'tin_hl_vs_tin': [('precuneus', 'precuneus')]}

In [ ]:
barheights = [[x for x in valdict.values()][:-1] for filename,valdict in accDict.items()] 

In [ ]:
bheights = np.array(barheights).T

In [ ]:
color = ['red','green','blue']

In [ ]:
keys = list(x.rstrip('tv_linear') for x in accDict.keys())

In [ ]:
matplotlib.rcParams.update({'font.size': 40})
fig,ax = plt.subplots(figsize=(12,12))
ax.set_ylim(0,1.2)
ax.grid(True)
for i,filebar in enumerate(bheights):
    ax.bar([i/3+2*j for j,_ in enumerate(bheights[0])],filebar,color = color[i], 
           width = 1/3,tick_label = keys)
plt.xticks([1/3+2*j for j in range(0,6)], rotation=45)
plt.legend(['Riemann','SVM corr','SVM lag corr'])
plt.ylabel('Accuracy')
plt.title('Classifier Accuracies')
dx = -60/72.; dy = 0/72. 
offset = matplotlib.transforms.ScaledTranslation(dx, dy, fig.dpi_scale_trans)

for label in ax.xaxis.get_majorticklabels():
    label.set_transform(label.get_transform() + offset)
    
plt.savefig('RiemannResults/Graphs/ClassAcc.png',bbox_inches='tight')

### Now use tin thresholds in class definitions

In [ ]:
accDict = {}
simDict = {}
matDict = {}
corrDict = {}
for fname in fnames:
    with gzip.open(data_dir+fname + '.bin', 'rb') as stream:
        data = pickle.load(stream)
        
    simArr = []
        
    # get time series data to make covariance matrices
    X = np.array([sample['TimeSeries'] for sample in data.samples if data_selector(sample)])        
    y = np.array([get_label_4_tin_thresh(sample['Group'],sample['Name'],metadata) for sample in data.samples if data_selector(sample)])
    
    # gsr seems to produce a rank deficient covariance matrix, so oas regularization is necessary
    if fname.startswith('no'):
        covest = Covariances()
    else:
        covest = Covariances('oas')
    ts = TangentSpace()
    svc = SVC(kernel='linear')
    clf_riem = make_pipeline(covest,ts,svc)
        
    X_base = np.array([sample['CRM'][np.triu_indices(sample['CRM'].shape[0])].flatten() for sample in data.samples if data_selector(sample)])
    y_base = np.array([get_label_4_tin_thresh(sample['Group'],sample['Name'],metadata) for sample in data.samples if data_selector(sample)])
    svc_base = SVC(kernel='linear')
    clf_base = svc_base
    
    X_lag = np.array([sample['LCM'][np.triu_indices(sample['LCM'].shape[0])].flatten() for sample in data.samples if data_selector(sample)])
    y_lag = np.array([get_label_4_tin_thresh(sample['Group'],sample['Name'],metadata) for sample in data.samples if data_selector(sample)])
    svc_lag = SVC(kernel='linear')
    clf_lag = svc_lag
    
    clf_composite = SVC(kernel='linear')
    
    #Check clustering
    #to_TS = make_pipeline(covest,ts)
    #X_in_TS = to_TS.transform(X)
    #kmeans = KMeans(n_clusters=4,random_state=0).fit(X_in_TS)
            
    # Monte Carlo, in theory should run this len(y)^2 times, but I need to save my poor computer's memory.
    accRiemList = []
    accBaseList = []
    accLagList = []
    accCompList = []
    coeffArr = []
    matRiemList = []
    matBaseList = []
    matLagList = []
    corrArrBefore = []
    corrArrAfter = []

    rs = StratifiedShuffleSplit(n_splits=100, test_size=.3)
    for i,(train_inds,test_inds) in enumerate(rs.split(X,y)):
        X_train, X_test, y_train, y_test = X[train_inds],X[test_inds],y[train_inds],y[test_inds]
        X_train_base, X_test_base, y_train_base, y_test_base = X_base[train_inds],X_base[test_inds],y_base[train_inds],y_base[test_inds]
        X_train_lag, X_test_lag, y_train_lag, y_test_lag = X_lag[train_inds],X_lag[test_inds],y_lag[train_inds],y_lag[test_inds]
        
        clf_riem.fit(X_train,y_train)
        clf_base.fit(X_train_base,y_train_base)
        clf_lag.fit(X_train_lag,y_train_lag)
        clf_composite.fit(X_train_lag,y_train_lag)
        
        #get riemann svm coefficients
        coeffArr.append(clf_riem[2].coef_)
            
        #compare correlation
        corr_coeffs_before = np.corrcoef(np.vstack([x[np.triu_indices(33)].flatten() for x in X_train]),rowvar=False)
        corrArrBefore.append(np.linalg.norm(corr_coeffs_before))
        
        ref = ts.reference_
        covs = covest.transform(X_train)
        mapped = ts.transform(covs)
        corr_coeffs_after = np.corrcoef(mapped,rowvar=False)
        corrArrAfter.append(np.linalg.norm(corr_coeffs_after))
        
        
        #for flattened in mapped:
            
            #mat = np.zeros((33,33))
            #mat[np.triu_indices(33)] = flattened
            #mat = (mat + mat.T)/2
            #mapped_mats.append(mat)
            
        #print(np.mean(np.array([distance_riemann(x,ref) for x in covs])-np.array([np.linalg.norm(x,ord='fro') for x in mapped_mats])/np.array([distance_riemann(x,ref) for x in covs])))
        
        y_pred = clf_riem.predict(X_test)
        y_pred_base = clf_base.predict(X_test_base)
        y_pred_lag = clf_lag.predict(X_test_lag)
        
        # save accuracy
        accRiemList.append(accuracy_score(y_pred,y_test))
        accBaseList.append(accuracy_score(y_pred_base,y_test_base))
        accLagList.append(accuracy_score(y_pred_lag,y_test_lag))
        accCompList.append(accuracy_score(clf_composite.predict(X_test_base),y_test_base))
        
        # confusion matrix
        mat = confusion_matrix(y_test,y_pred,normalize='true',labels=[0,1,2,3])
        #plot_confusion_matrix(clf_riem, X_test, y_test,normalize='true',labels=['ctr','hl','tin_hl','tin'],display_labels=['ctr','ctr_hl','hl_tin','tin'])
        #plt.savefig('RiemannResults/' + fname + '/ConfMatRiemRun' + str(i) + '.png',bbox_inches='tight')
        #plt.close()
        mat_base = confusion_matrix(y_test_base,y_pred_base,normalize='true',labels=[0,1,2,3])
        #plot_confusion_matrix(clf_base, X_test_base, y_test_base,normalize='true',labels=['ctr','hl','tin_hl','tin'],display_labels=['ctr','ctr_hl','hl_tin','tin'])
        #plt.savefig('RiemannResults/' + fname + '/ConfMatBaseRun' + str(i) + '.png',bbox_inches='tight')
        #plt.close()
        mat_lag = confusion_matrix(y_test_lag,y_pred_lag,normalize='true',labels=[0,1,2,3])
        #plot_confusion_matrix(clf_lag, X_test_lag, y_test_lag,normalize='true',labels=['ctr','hl','tin_hl','tin'],display_labels=['ctr','ctr_hl','hl_tin','tin'])
        #plt.savefig('RiemannResults/' + fname + '/ConfMatLagRun' + str(i) + '.png',bbox_inches='tight')
        #plt.close()
        
        matRiemList.append(mat)
        matBaseList.append(mat_base)
        matLagList.append(mat_lag)
    
    for z in range(0,len(coeffArr[0])):
        class_z_coeffs = [x[z] for x in coeffArr]
        cos_sim = cosine_similarity(class_z_coeffs)
        upperTri = cos_sim[np.triu_indices(cos_sim.shape[0],1)]
        cos_avg = np.mean(upperTri.flatten())
        simArr.append(cos_avg)
        
    avgMatRiem = sum(matRiemList)/len(matRiemList)
    simDict.update({fname: simArr}) 
    matDict.update({fname: avgMatRiem})
    riemAcc = np.mean(accRiemList)
    baseAcc = np.mean(accBaseList)
    lagAcc = np.mean(accLagList)
    compAcc = np.mean(accCompList)
    accDict.update({fname:{'riem':riemAcc, 'base': baseAcc,'lag': lagAcc,'comp':compAcc}})
    corrDict.update({fname:{'before':np.mean(corrArrBefore),'after':np.mean(corrArrAfter)}})        
    print("Mean Accuracy w/ Riemann on file " + fname + ": " + str(riemAcc))
    print("Mean Accuracy w/ Base on file " + fname + ": " + str(baseAcc))
    print("Mean Accuracy w/ Lag on file " + fname + ": " + str(lagAcc))
    print("Mean Accuracy w/ Composite on file " + fname + ": " + str(compAcc))
    print("----------------")


In [ ]:
accDict = {}
simDict = {}
matDict = {}
corrDict = {}
for fname in fnames:
    with gzip.open(data_dir+fname + '.bin', 'rb') as stream:
        data = pickle.load(stream)
        
    simArr = []
        
    # get time series data to make covariance matrices
    X = np.array([sample['TimeSeries'] for sample in data.samples if data_selector(sample)])        
    y = np.array([get_label_4_tin_thresh2(sample['Group'],sample['Name'],metadata) for sample in data.samples if data_selector(sample)])
    
    # gsr seems to produce a rank deficient covariance matrix, so oas regularization is necessary
    if fname.startswith('no'):
        covest = Covariances()
    else:
        covest = Covariances('oas')
    ts = TangentSpace()
    svc = SVC(kernel='linear')
    clf_riem = make_pipeline(covest,ts,svc)
        
    X_base = np.array([sample['CRM'][np.triu_indices(sample['CRM'].shape[0])].flatten() for sample in data.samples if data_selector(sample)])
    y_base = np.array([get_label_4_tin_thresh2(sample['Group'],sample['Name'],metadata) for sample in data.samples if data_selector(sample)])
    svc_base = SVC(kernel='linear')
    clf_base = svc_base
    
    X_lag = np.array([sample['LCM'][np.triu_indices(sample['LCM'].shape[0])].flatten() for sample in data.samples if data_selector(sample)])
    y_lag = np.array([get_label_4_tin_thresh2(sample['Group'],sample['Name'],metadata) for sample in data.samples if data_selector(sample)])
    svc_lag = SVC(kernel='linear')
    clf_lag = svc_lag
    
    clf_composite = SVC(kernel='linear')
    
    #Check clustering
    #to_TS = make_pipeline(covest,ts)
    #X_in_TS = to_TS.transform(X)
    #kmeans = KMeans(n_clusters=4,random_state=0).fit(X_in_TS)
            
    # Monte Carlo, in theory should run this len(y)^2 times, but I need to save my poor computer's memory.
    accRiemList = []
    accBaseList = []
    accLagList = []
    accCompList = []
    coeffArr = []
    matRiemList = []
    matBaseList = []
    matLagList = []
    corrArrBefore = []
    corrArrAfter = []

    rs = StratifiedShuffleSplit(n_splits=100, test_size=.3)
    for i,(train_inds,test_inds) in enumerate(rs.split(X,y)):
        X_train, X_test, y_train, y_test = X[train_inds],X[test_inds],y[train_inds],y[test_inds]
        X_train_base, X_test_base, y_train_base, y_test_base = X_base[train_inds],X_base[test_inds],y_base[train_inds],y_base[test_inds]
        X_train_lag, X_test_lag, y_train_lag, y_test_lag = X_lag[train_inds],X_lag[test_inds],y_lag[train_inds],y_lag[test_inds]
        
        clf_riem.fit(X_train,y_train)
        clf_base.fit(X_train_base,y_train_base)
        clf_lag.fit(X_train_lag,y_train_lag)
        clf_composite.fit(X_train_lag,y_train_lag)
        
        #get riemann svm coefficients
        coeffArr.append(clf_riem[2].coef_)
            
        #compare correlation
        corr_coeffs_before = np.corrcoef(np.vstack([x[np.triu_indices(33)].flatten() for x in X_train]),rowvar=False)
        corrArrBefore.append(np.linalg.norm(corr_coeffs_before))
        
        ref = ts.reference_
        covs = covest.transform(X_train)
        mapped = ts.transform(covs)
        corr_coeffs_after = np.corrcoef(mapped,rowvar=False)
        corrArrAfter.append(np.linalg.norm(corr_coeffs_after))
        
        
        #for flattened in mapped:
            
            #mat = np.zeros((33,33))
            #mat[np.triu_indices(33)] = flattened
            #mat = (mat + mat.T)/2
            #mapped_mats.append(mat)
            
        #print(np.mean(np.array([distance_riemann(x,ref) for x in covs])-np.array([np.linalg.norm(x,ord='fro') for x in mapped_mats])/np.array([distance_riemann(x,ref) for x in covs])))
        
        y_pred = clf_riem.predict(X_test)
        y_pred_base = clf_base.predict(X_test_base)
        y_pred_lag = clf_lag.predict(X_test_lag)
        
        # save accuracy
        accRiemList.append(accuracy_score(y_pred,y_test))
        accBaseList.append(accuracy_score(y_pred_base,y_test_base))
        accLagList.append(accuracy_score(y_pred_lag,y_test_lag))
        accCompList.append(accuracy_score(clf_composite.predict(X_test_base),y_test_base))
        
        # confusion matrix
        mat = confusion_matrix(y_test,y_pred,normalize='true',labels=[0,1,2,3])
        #plot_confusion_matrix(clf_riem, X_test, y_test,normalize='true',labels=['ctr','hl','tin_hl','tin'],display_labels=['ctr','ctr_hl','hl_tin','tin'])
        #plt.savefig('RiemannResults/' + fname + '/ConfMatRiemRun' + str(i) + '.png',bbox_inches='tight')
        #plt.close()
        mat_base = confusion_matrix(y_test_base,y_pred_base,normalize='true',labels=[0,1,2,3])
        #plot_confusion_matrix(clf_base, X_test_base, y_test_base,normalize='true',labels=['ctr','hl','tin_hl','tin'],display_labels=['ctr','ctr_hl','hl_tin','tin'])
        #plt.savefig('RiemannResults/' + fname + '/ConfMatBaseRun' + str(i) + '.png',bbox_inches='tight')
        #plt.close()
        mat_lag = confusion_matrix(y_test_lag,y_pred_lag,normalize='true',labels=[0,1,2,3])
        #plot_confusion_matrix(clf_lag, X_test_lag, y_test_lag,normalize='true',labels=['ctr','hl','tin_hl','tin'],display_labels=['ctr','ctr_hl','hl_tin','tin'])
        #plt.savefig('RiemannResults/' + fname + '/ConfMatLagRun' + str(i) + '.png',bbox_inches='tight')
        #plt.close()
        
        matRiemList.append(mat)
        matBaseList.append(mat_base)
        matLagList.append(mat_lag)
    
    for z in range(0,len(coeffArr[0])):
        class_z_coeffs = [x[z] for x in coeffArr]
        cos_sim = cosine_similarity(class_z_coeffs)
        upperTri = cos_sim[np.triu_indices(cos_sim.shape[0],1)]
        cos_avg = np.mean(upperTri.flatten())
        simArr.append(cos_avg)
        
    avgMatRiem = sum(matRiemList)/len(matRiemList)
    simDict.update({fname: simArr}) 
    matDict.update({fname: avgMatRiem})
    riemAcc = np.mean(accRiemList)
    baseAcc = np.mean(accBaseList)
    lagAcc = np.mean(accLagList)
    compAcc = np.mean(accCompList)
    accDict.update({fname:{'riem':riemAcc, 'base': baseAcc,'lag': lagAcc,'comp':compAcc}})
    corrDict.update({fname:{'before':np.mean(corrArrBefore),'after':np.mean(corrArrAfter)}})        
    print("Mean Accuracy w/ Riemann on file " + fname + ": " + str(riemAcc))
    print("Mean Accuracy w/ Base on file " + fname + ": " + str(baseAcc))
    print("Mean Accuracy w/ Lag on file " + fname + ": " + str(lagAcc))
    print("Mean Accuracy w/ Composite on file " + fname + ": " + str(compAcc))
    print("----------------")